<a href="https://colab.research.google.com/github/LeeMinJun0102/Car_Price_Predict/blob/main/DL_test1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [ ]:
!pip install -q tensorflow opencv-python-headless scikit-learn matplotlib pillow

import os
import shutil
import matplotlib.pyplot as plt
import cv2

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import warnings
from google.colab import files
warnings.filterwarnings('ignore')

print("✅ 모든 라이브러리 설치 및 import 완료!")

# 1 colab drive 연결

In [ ]:
from google.colab import drive
print("🔗 Google Drive에 연결 중...")
drive.mount('/content/drive')

# 2 폴더 생성

In [ ]:
# 1. GPU 런타임 확인
print("TensorFlow 버전:", tf.__version__)
print("GPU 사용 가능:", tf.config.list_physical_devices('GPU'))

# 2. 폴더 구조 생성
os.makedirs('car_data/train', exist_ok=True)
os.makedirs('car_data/test', exist_ok=True)
os.makedirs('car_data/models', exist_ok=True)

print("✅ 폴더 구조 생성 완료!")
print("다음 단계: 이미지 데이터를 업로드하세요.")

In [ ]:
# google drive 연결
def google_drive():
    # Drive 내 car_data 폴더 경로
    source_path = '/content/drive/MyDrive/itwill_Final_Project/car_data'

    if source_path:
        print(f"📁 Drive 내 데이터 위치 : {source_path}")

        # 빠른 처리를 위해 로컬로 복사
        if os.path.exists('./car_data'):
            shutil.rmtree('./car_data')

        print("📋 데이터를 로컬로 복사 중... (시간이 소요될 수 있습니다)")
        shutil.copytree(source_path, './car_data')
        print("✅ 데이터 복사 완료!")

    else:
        print("❌ Google Drive에서 car_data 폴더를 찾을 수 없습니다.")

    check_data_structure()
    return True

# 데이터 구조 확인 함수
def check_data_structure():
    """업로드된 데이터 구조 확인"""
    print("\n📊 업로드된 데이터 구조 확인:")

    train_dir = 'car_data/train'

    if not os.path.exists(train_dir):
        print("❌ car_data/train 폴더가 없습니다!")
        return False

    total_images = 0
    brands = []

    for brand in os.listdir(train_dir):
        brand_path = os.path.join(train_dir, brand)

        if os.path.isdir(brand_path):
            image_files = [f for f in os.listdir(brand_path)
                           if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
            image_count = len(image_files)

            print(f"  📁 {brand}: {image_count}장")
            total_images += image_count
            brands.append(brand)

    print(f"\n✅ 총 {len(brands)}개 브랜드, {total_images}장의 이미지")

    # 최소 요구사항 체크
    if len(brands) < 3:
        print("⚠️ 최소 3개 이상의 브랜드가 필요합니다.")

    if total_images < 300:
        print("⚠️ 전체 이미지가 300장 미만입니다. 더 많은 데이터가 권장됩니다.")

    return total_images > 0

print("Google Drive 연결")
google_drive()

# 3 이미지 로드 및 전처리

In [ ]:
def complete_data_processor_enhanced(data_dir, img_size=(224, 224)):
    """향상된 전처리가 포함된 데이터 처리"""

    print("\n📊 데이터 구조 확인 및 처리 중...")

    if not os.path.exists(data_dir):
        print("❌ 데이터 폴더가 없습니다!")
        return None, None, None

    images = []
    labels = []
    brand_info = {}
    sample_images = {}

    # 한 번의 순회로 모든 작업 처리
    for brand in os.listdir(data_dir):
        brand_path = os.path.join(data_dir, brand)

        if os.path.isdir(brand_path):
            print(f"  📁 {brand} 처리 중...")
            brand_count = 0
            first_image_for_display = None

            for img_file in os.listdir(brand_path):
                if img_file.lower().endswith(('.png', '.jpg', '.jpeg')):
                    img_path = os.path.join(brand_path, img_file)

                    try:
                        image = cv2.imread(img_path)
                        if image is not None:
                            # BGR to RGB 변환
                            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

                            # 첫 번째 이미지는 샘플 표시용으로 저장 (전처리 전)
                            if first_image_for_display is None:
                                first_image_for_display = image_rgb.copy()

                            # 히스토그램 평활화 (조명 정규화)
                            lab = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2LAB)
                            lab[:,:,0] = cv2.equalizeHist(lab[:,:,0])  # L 채널만 평활화
                            image_rgb = cv2.cvtColor(lab, cv2.COLOR_LAB2RGB)

                            # 모델용 전처리 (리사이징 + 정규화)
                            image_processed = cv2.resize(image_rgb, img_size)
                            image_processed = image_processed.astype(np.float32) / 255.0

                            images.append(image_processed)
                            labels.append(brand)
                            brand_count += 1

                    except Exception as e:
                        print(f"    ⚠️ 이미지 처리 실패 {img_file}: {e}")

            # 브랜드별 정보 저장
            if brand_count > 0:
                brand_info[brand] = brand_count
                if first_image_for_display is not None:
                    sample_images[brand] = first_image_for_display

            print(f"    ✅ {brand}: {brand_count}장 완료")

    # 데이터 구조 리포트
    total_images = sum(brand_info.values())
    print(f"\n✅ 총 {len(brand_info)}개 브랜드, {total_images}장의 이미지")

    # 최소 요구사항 체크
    if len(brand_info) < 3:
        print("⚠️ 최소 3개 이상의 브랜드가 필요합니다.")
    if total_images < 300:
        print("⚠️ 전체 이미지가 300장 미만입니다.")

    # 샘플 이미지 표시
    if sample_images:
        display_samples_unified(sample_images, brand_info)

    # 데이터 처리 완료
    if len(images) > 0:
        images = np.array(images)
        label_encoder = LabelEncoder()
        encoded_labels = label_encoder.fit_transform(labels)
        categorical_labels = to_categorical(encoded_labels)

        print(f"✅ 모든 처리 완료: {len(images)}장, {len(label_encoder.classes_)}개 클래스")
        print(f"✅ 클래스: {list(label_encoder.classes_)}")

        return images, categorical_labels, label_encoder.classes_

    return None, None, None

def display_samples_unified(sample_images, brand_info):
    """통합된 샘플 이미지 표시"""
    brands = list(sample_images.keys())
    n_show = min(6, len(brands))
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.flatten()

    for i, brand in enumerate(brands[:n_show]):
        axes[i].imshow(sample_images[brand])
        axes[i].set_title(f'{brand}\n(count : {brand_info[brand]})')
        axes[i].axis('off')

    for i in range(n_show, 6):
        axes[i].axis('off')

    plt.suptitle('Data Structure & Samples', fontsize=16)
    plt.tight_layout()
    plt.show()

# 실행
print("2️⃣ Google Drive 연결")
google_drive()

# 향상된 전처리 적용
images, labels, class_names = complete_data_processor_enhanced('car_data/train')

if images is not None:
    print("\n🎉 향상된 전처리로 모든 처리 완료! 모델 학습 준비됨")

# 4 모델 생성

In [ ]:
def create_car_model(num_classes, learning_rate=0.0001):
    print("모델 생성 중...")

    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    # 레이어 동결 (마지막 30개만 학습)
    for layer in base_model.layers[:-30]:
        layer.trainable = False

    model = tf.keras.Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(512, activation='relu', name='feature_dense'),    # 이름 추가
        Dropout(0.5),
        Dense(256, activation='relu'),
        Dropout(0.3),
        Dense(num_classes, activation='softmax', name='predictions')    # 이름 추가
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate),
        loss='categorical_crossentropy',
        metrics=['accuracy', 'top_k_categorical_accuracy']  # 메트릭스 추가
    )

    print(f"모델 완성: {len(class_names)}개 클래스")
    return model

if 'class_names' in locals() and class_names is not None:
    model = create_car_model(num_classes=len(class_names))
    model.summary()
else:
    print('데이터를 먼저 로드해주세요')

# 5 모델 학습 및 성능 평가

In [ ]:
def simple_train_model(model, images, labels, epochs=25):
    """간단한 모델 훈련"""
    print('훈련 시작!')

    # 데이터 분할
    X_train, X_val, y_train, y_val = train_test_split(
        images, labels, test_size=0.2, random_state=42,
        stratify=labels
    )

    # 데이터 증강
    train_gen = ImageDataGenerator(
        rotation_range=15, width_shift_range=0.1, height_shift_range=0.1,
        zoom_range=0.15, horizontal_flip=True, brightness_range=[0.8, 1.2]
    )
    val_gen = ImageDataGenerator()

    # 콜백
    callbacks = [
        tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=8, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4),
        tf.keras.callbacks.ModelCheckpoint('car_data/models/best_car_model.h5', monitor='val_accuracy', save_best_only=True)
    ]

    # 훈련
    history = model.fit(
        train_gen.flow(X_train, y_train, batch_size=32),
        steps_per_epoch=len(X_train) // 32,
        epochs=epochs,
        validation_data=val_gen.flow(X_val, y_val, batch_size=32),
        validation_steps=len(X_val) // 32,
        callbacks=callbacks
    )

    # 시각화
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))

    axes[0,0].plot(history.history['accuracy'], label='Train')
    axes[0,0].plot(history.history['val_accuracy'], label='Val')
    axes[0,0].set_title('Accuracy')
    axes[0,0].legend()

    axes[0,1].plot(history.history['loss'], label='Train')
    axes[0,1].plot(history.history['val_loss'], label='Val')
    axes[0,1].set_title('Loss')
    axes[0,1].legend()

    if 'top_k_categorical_accuracy' in history.history:
        axes[1,0].plot(history.history['top_k_categorical_accuracy'], label='Train')
        axes[1,0].plot(history.history['val_top_k_categorical_accuracy'], label='Val')
        axes[1,0].set_title('Top-K Accuracy')
        axes[1,0].legend()

    axes[1,1].axis('off')
    plt.tight_layout()
    plt.show()

    # 결과 출력
    final_acc = max(history.history['val_accuracy'])
    print(f"최고 검증 정확도: {final_acc:.4f}")

    return history, X_val, y_val

# 실행
if 'model' in locals() and 'images' in locals():
    history, X_val, y_val = simple_train_model(model, images, labels)
else:
    print("모델과 데이터를 먼저 준비해주세요.")

# 6 자동차 이미지 유사도 검색

In [ ]:
def create_search_system(model, class_names):
    # Sequential 모델에서 'feature_dense' 레이어 찾기
    feature_layer = None
    for layer in model.layers:
        if hasattr(layer, 'name') and layer.name == 'feature_dense':
            feature_layer = layer
            break

    if feature_layer is None:
        # 대안: Dense 512 레이어 찾기
        for layer in model.layers:
            if hasattr(layer, 'units') and layer.units == 512:
                feature_layer = layer
                break

    # feature_dense 레이어까지의 모델 생성
    feature_model = tf.keras.Sequential()
    for layer in model.layers:
        feature_model.add(layer)
        if layer == feature_layer:
            break

    return feature_model

def build_database(feature_model, images, labels):
    print("데이터베이스 구축 중...")
    db_features = feature_model.predict(images, verbose=0)
    db_labels = np.argmax(labels, axis=1)
    print(f"완료: {len(images)}개 이미지")
    return db_features, db_labels, images

def search_similar(query_image, feature_model, db_features, db_labels, db_images, class_names, top_k=11):
    # 쿼리 이미지 특징 추출
    if len(query_image.shape) == 3:
        query_image = np.expand_dims(query_image, 0)
    query_features = feature_model.predict(query_image, verbose=0)

    # 유사도 계산
    similarities = cosine_similarity(query_features, db_features)[0]
    top_idx = np.argsort(similarities)[::-1][:top_k]

    # 결과
    results = []
    for i, idx in enumerate(top_idx):
        results.append({
            'rank': i+1,
            'car': class_names[db_labels[idx]],
            'similarity': similarities[idx],
            'image': db_images[idx]
        })
    return results

def show_results(query_image, results):
    fig, axes = plt.subplots(4, 3, figsize=(15, 8))
    axes = axes.flatten()

    axes[0].imshow(query_image.squeeze())
    axes[0].set_title('Query')
    axes[0].axis('off')

    for i, r in enumerate(results[:11]):
        axes[i+1].imshow(r['image'])
        axes[i+1].set_title(f"{r['rank']}. {r['car']}\n{r['similarity']:.3f}")
        axes[i+1].axis('off')

    plt.tight_layout()
    plt.show()

# 사용법
if 'model' in locals() and 'images' in locals():
    feature_model = create_search_system(model, class_names)
    db_features, db_labels, db_images = build_database(feature_model, images, labels)

    query = images[0]
    results = search_similar(query, feature_model, db_features, db_labels, db_images, class_names)
    show_results(query, results)

    for r in results:
        print(f"{r['rank']}위: {r['car']} (유사도: {r['similarity']:.3f})")

# 7 이미지 테스트

In [ ]:
def upload_test():
    """향상된 전처리가 포함된 업로드 테스트"""
    from google.colab import files
    uploaded = files.upload()

    if uploaded:
        filename = list(uploaded.keys())[0]
        img = cv2.imread(filename)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # 학습 데이터와 동일한 전처리 적용
        lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
        lab[:,:,0] = cv2.equalizeHist(lab[:,:,0])
        img = cv2.cvtColor(lab, cv2.COLOR_LAB2RGB)

        processed = cv2.resize(img, (224, 224)).astype(np.float32) / 255.0

        # 검색 및 결과 표시
        results = search_similar(processed, feature_model, db_features, db_labels, db_images, class_names)
        show_results(processed, results)

        print(f"가장 유사한 차량: {results[0]['car']}")
        return processed
    return None

def batch_test(test_images, test_labels):
    """일괄 평가"""
    correct_top1 = correct_top3 = 0

    for img, label in zip(test_images, test_labels):
        true_class = class_names[np.argmax(label)]
        results = search_similar(img, feature_model, db_features, db_labels, db_images, class_names)

        pred_classes = [r['car'] for r in results]
        if true_class == pred_classes[0]:
            correct_top1 += 1
        if true_class in pred_classes[:3]:
            correct_top3 += 1

    total = len(test_images)
    print(f"Top-1 정확도: {correct_top1/total:.3f}")
    print(f"Top-3 정확도: {correct_top3/total:.3f}")

# 사용법
if 'feature_model' in locals() and 'db_features' in locals():
    # 업로드 테스트
    upload_test()

    # 검증 데이터 평가
    if 'X_val' in locals():
        batch_test(X_val[:20], y_val[:20])